# Chapter 12: Sequence-to-Sequence Learning: Part 2 (Attention)

## 1️⃣ Chapter Overview

In Chapter 11, we implemented a standard Encoder–Decoder Seq2Seq model for machine translation. While effective for short sequences, this architecture suffers from a fundamental limitation known as the **information bottleneck**: the encoder must compress the entire source sentence into a single fixed-size context vector.

As sentence length increases, this bottleneck leads to information loss, degraded long-range dependency modeling, and poor translation quality. This chapter introduces the **Attention Mechanism**, specifically **Bahdanau Attention (Additive Attention)**, as a principled solution to this problem.

Instead of relying on a single static context vector, Attention allows the decoder to dynamically access **all encoder hidden states** at every decoding step. This enables the model to selectively focus on the most relevant source words when generating each target token.

Beyond improving performance, attention also provides a degree of **model interpretability**, as attention weights can be visualized to reveal soft alignments between source and target tokens.

---


## 2️⃣ Theoretical Explanation

### 2.1 The Bottleneck Problem in Vanilla Seq2Seq Models

In a standard Seq2Seq model without attention, the encoder processes the source sequence $X = (x_1, x_2, ..., x_T)$ and produces a sequence of hidden states. However, only the **final hidden state** $h_T$ is passed to the decoder as the context vector.

This design forces the model to encode all semantic, syntactic, and contextual information of the source sentence into a vector of fixed dimensionality, regardless of sentence length. Consequently, short sentences and very long sentences are represented using vectors of identical size.

As sequence length grows, the encoder must compress increasingly complex information into the same representational capacity. This results in information loss and causes the decoder to struggle with long sentences, rare words, and long-range dependencies. Empirically, this manifests as degraded translation quality for longer inputs.

---


### 2.2 Attention as a Solution to the Bottleneck

The Attention mechanism eliminates the fixed-context bottleneck by allowing the decoder to access **all encoder hidden states** $[h_1, h_2, ..., h_T]$ at each decoding timestep. Instead of relying on a single summary vector, the decoder computes a **dynamic context vector** that changes at every timestep.

At decoding step $t$, attention answers the question: *Which parts of the source sentence are most relevant for generating the next target token?* To do this, the model computes a relevance score between the decoder’s previous hidden state and each encoder hidden state.

This process consists of three conceptual stages: scoring, weighting, and context aggregation. Together, these steps allow the model to softly align target tokens with source tokens during generation.

---


At each decoder timestep $t$, the attention mechanism performs the following computations:

1. **Alignment Score Computation**: For each encoder hidden state $h_j$, the model computes an alignment score with the decoder’s previous hidden state $s_{t-1}$. This score reflects how relevant the source position $j$ is for predicting the next target token.

2. **Attention Weight Normalization**: The alignment scores are normalized using the Softmax function to produce attention weights $\alpha_{tj}$, which form a probability distribution over source positions:

$$ \alpha_{tj} = \frac{\exp(\text{score}(s_{t-1}, h_j))}{\sum_{k=1}^{T} \exp(\text{score}(s_{t-1}, h_k))} $$

3. **Context Vector Construction**: The dynamic context vector for timestep $t$ is computed as a weighted sum of encoder hidden states:

$$ c_t = \sum_{j=1}^{T} \alpha_{tj} h_j $$

This context vector captures the most relevant source-side information needed to generate the next target token.

---


### 2.3 Bahdanau Attention (Additive Attention)

Bahdanau Attention, also known as **Additive Attention**, was proposed by Bahdanau et al. (2014) and represents one of the earliest and most influential attention mechanisms for neural machine translation.

In Bahdanau Attention, the alignment score between the decoder’s previous hidden state $s_{t-1}$ and an encoder hidden state $h_j$ is computed using a small feed-forward neural network:

$$ \text{score}(s_{t-1}, h_j) = v_a^T \tanh(W_a s_{t-1} + U_a h_j) $$

Here, $W_a$, $U_a$, and $v_a$ are learnable parameters. Unlike dot-product attention, this formulation allows the model to learn a nonlinear compatibility function between encoder and decoder states.

Bahdanau Attention is particularly effective when encoder and decoder hidden states have different dimensionalities, making it well-suited for RNN-based Seq2Seq models.

---


### 2.4 Decoder with Attention Integration

When attention is incorporated into the decoder, the decoder’s hidden state update depends not only on the previous target token and hidden state, but also on the dynamically computed context vector $c_t$.

The decoder state update can be expressed as:

$$ s_t = f(s_{t-1}, y_{t-1}, c_t) $$

The output distribution over the target vocabulary is then computed using both the decoder state and the context vector. This allows the decoder to condition each prediction on the most relevant parts of the source sentence, effectively mitigating long-sequence degradation.

From an architectural perspective, attention transforms the encoder from a static information source into a **memory bank** that the decoder can query at every timestep.

---


## 3️⃣ Attention and Model Interpretability

A key advantage of attention mechanisms is improved **interpretability**. The attention weights $\alpha_{tj}$ provide a soft alignment between source and target tokens, indicating which source words the model attends to when generating each target word.

By visualizing attention weights as heatmaps, it is possible to analyze word-to-word correspondences, diagnose translation errors, and gain insight into the model’s decision-making process. Although attention does not guarantee perfect linguistic alignment, it offers a valuable window into otherwise opaque neural sequence models.

---


## Chapter Summary

This chapter extends the Seq2Seq framework by introducing attention mechanisms to address the fixed-context bottleneck of vanilla encoder–decoder models. By computing a dynamic context vector at each decoding step, attention allows the decoder to selectively focus on relevant source tokens and significantly improves performance on long sequences.

Bahdanau Attention provides a flexible, learnable alignment function that integrates naturally with RNN-based Seq2Seq models. Beyond performance gains, attention also enhances interpretability by exposing soft alignments between source and target sequences.

These ideas form the conceptual foundation for **Transformer models**, which replace recurrent architectures entirely with attention-based mechanisms, as explored in later chapters.


## 3️⃣ Data Preparation

We will reuse the English-German translation dataset preparation steps from Chapter 11. We load the data, clean it, vectorization it, and prepare the `tf.data` pipeline.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import pandas as pd
import zipfile

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# --- 1. Data Loading (Same as Chapter 11) ---
url = "http://www.manythings.org/anki/deu-eng.zip"
zip_path = tf.keras.utils.get_file("deu-eng.zip", origin=url, extract=True)
text_file = os.path.join(os.path.dirname(zip_path), "deu.txt")

def load_data(path, num_samples=50000):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    data = []
    for line in lines[:min(num_samples, len(lines)-1)]:
        parts = line.split('\t')
        if len(parts) >= 2:
            english = parts[0]
            german = f"sos {parts[1]} eos" # Start/End tokens
            data.append([english, german])
    return np.array(data)

raw_data = load_data(text_file)
print(f"Loaded {len(raw_data)} samples.")

# --- 2. Train/Val/Test Split ---
indices = np.arange(len(raw_data))
np.random.shuffle(indices)
raw_data = raw_data[indices]

num_val = int(0.1 * len(raw_data))
num_test = int(0.1 * len(raw_data))
train_pairs = raw_data[:-(num_val + num_test)]
val_pairs = raw_data[-(num_val + num_test):-num_test]
test_pairs = raw_data[-num_test:]

# --- 3. Text Vectorization ---
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 20

en_vectorizer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=SEQUENCE_LENGTH,
    standardize='lower_and_strip_punctuation')

de_vectorizer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=SEQUENCE_LENGTH + 1,
    standardize='lower_and_strip_punctuation')

en_vectorizer.adapt(train_pairs[:, 0])
de_vectorizer.adapt(train_pairs[:, 1])

# --- 4. Dataset Pipeline ---
def format_dataset(eng, deu):
    eng = en_vectorizer(eng)
    deu = de_vectorizer(deu)
    return ({'encoder_inputs': eng, 'decoder_inputs': deu[:, :-1]}, deu[:, 1:])

def make_dataset(pairs, batch_size=64):
    eng_texts, deu_texts = pairs[:, 0], pairs[:, 1]
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, deu_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

## 4️⃣ Custom Attention Layer

This is the core contribution of this chapter. Since Keras did not natively support a simple plug-and-play Bahdanau Attention wrapper for RNN cells at the time of writing, we implement it manually.

We define `DecoderRNNAttentionWrapper`. This layer acts as a wrapper around a standard GRU cell. Inside its `call` method, it calculates the attention context and feeds the concatenated `[input, context]` to the GRU cell.

In [ ]:
class DecoderRNNAttentionWrapper(layers.Layer):
    def __init__(self, cell_fn, units, **kwargs):
        super(DecoderRNNAttentionWrapper, self).__init__(**kwargs)
        self._cell_fn = cell_fn  # The basic GRU Cell
        self.units = units       # Dimension of Attention internal layers

    def build(self, input_shape):
        # input_shape comes as [Encoder_Out_Shape, Decoder_Input_Shape]
        # Encoder Out Shape: (batch, time, hidden)
        
        # W_a: Weight for Encoder Outputs (Values)
        self.W_a = self.add_weight(
            name='W_a', 
            shape=tf.TensorShape((input_shape[0][2], self.units)),
            initializer='uniform', trainable=True)
        
        # U_a: Weight for Decoder State (Query)
        self.U_a = self.add_weight(
            name='U_a', 
            shape=tf.TensorShape((self._cell_fn.units, self.units)),
            initializer='uniform', trainable=True)
        
        # V_a: Weight for calculating the final score
        self.V_a = self.add_weight(
            name='V_a', 
            shape=tf.TensorShape((self.units, 1)),
            initializer='uniform', trainable=True)
        
        super(DecoderRNNAttentionWrapper, self).build(input_shape)

    def call(self, inputs, initial_state, training=False):
        # inputs is a list: [encoder_outputs, decoder_inputs]
        encoder_outputs, decoder_inputs = inputs
        
        # Define the step function for K.rnn
        # This function runs for every time step of the Decoder
        def _step(inputs, states):
            # inputs: Single decoder step input
            # states: List containing [prev_decoder_state, encoder_outputs]
            
            prev_decoder_state = states[0]
            full_encoder_outputs = states[-1] # Accessed via constants

            # --- Attention Score Calculation ---
            
            # 1. Transform Encoder Outputs: H * W_a 
            # Result shape: (batch, en_seq_len, units)
            W_a_dot_h = K.dot(full_encoder_outputs, self.W_a)
            
            # 2. Transform Previous Decoder State: S_{t-1} * U_a
            # Result shape: (batch, 1, units)
            U_a_dot_s = K.expand_dims(K.dot(prev_decoder_state, self.U_a), 1)
            
            # 3. Tanh Activation: tanh(W_a.H + U_a.S)
            # Broadcasting happens here to add (batch, 1, units) to (batch, seq, units)
            Wa_plus_Ua = K.tanh(W_a_dot_h + U_a_dot_s)
            
            # 4. Calculate Score: V_a^T * tanh(...)
            # Shape: (batch, en_seq_len, 1)
            e_i = K.dot(Wa_plus_Ua, self.V_a)
            # Squeeze to remove last dim -> (batch, en_seq_len)
            e_i = K.squeeze(e_i, axis=-1) 
            
            # 5. Softmax to get Attention Weights (alpha)
            a_i = K.softmax(e_i)
            
            # --- Context Vector Calculation ---
            # Weighted sum of encoder outputs
            # context = sum(alpha * H)
            c_i = K.sum(full_encoder_outputs * K.expand_dims(a_i, -1), axis=1)
            
            # --- GRU Step ---
            # Concatenate Context Vector + Current Decoder Input
            gru_input = K.concatenate([inputs, c_i], axis=-1)
            
            # Call the basic GRU cell
            output, new_states = self._cell_fn(gru_input, states)
            
            # Return: (Output, Attention_Weights), [New_State]
            return (output, a_i), new_states

        # K.rnn loops the _step function over the time dimension of decoder_inputs
        # constants=[encoder_outputs] ensures encoder_outputs is available in _step
        _, outputs, _ = K.rnn(
            step_function=_step,
            inputs=decoder_inputs,
            initial_states=[initial_state],
            constants=[encoder_outputs]
        )
        
        # Unpack outputs (GRU outputs, Attention Weights)
        decoder_outputs, attention_weights = outputs
        return decoder_outputs, attention_weights

### Step-by-Step Code Explanation

1.  **Input:** The layer receives `encoder_outputs` (the full history of the source sentence) and `decoder_inputs` (the target sentence offset by one).
2.  **`K.rnn` Loop:** Unlike standard layers that process matrices, we need to iterate step-by-step because the attention for time $t$ depends on the state from $t-1$. `K.rnn` handles this unrolling efficiently.
3.  **`_step` Function:** Inside the loop:
    * We project the encoder states (`W_a`) and the current decoder state (`U_a`) into a shared latent space `units`.
    * We apply `tanh` nonlinearity.
    * We project down to a score using `V_a`.
    * `K.softmax` converts scores into probabilities ($\{ \alpha_1, ..., \alpha_T \}$).
    * We compute the **Context Vector** as the weighted sum of encoder states.
    * We feed `[Decoder_Input, Context_Vector]` into the actual GRU cell.
4.  **Output:** The layer returns the GRU outputs (for prediction) AND the attention weights (for visualization).

## 5️⃣ Building the Attention-Based Seq2Seq Model

Now we integrate our custom layer into the functional Keras API.

In [ ]:
EMBEDDING_DIM = 128
LATENT_DIM = 256

# --- Encoder (Same as Chapter 11) ---
encoder_inputs = layers.Input(shape=(SEQUENCE_LENGTH,), dtype="int64", name="encoder_inputs")
enc_emb = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(encoder_inputs)
encoder_gru = layers.Bidirectional(layers.GRU(LATENT_DIM // 2, return_sequences=True, return_state=True))
encoder_out, fwd_state, bwd_state = encoder_gru(enc_emb)

# Concatenate forward and backward states for the initial decoder state
encoder_state = layers.Concatenate()([fwd_state, bwd_state])

# --- Decoder with Attention ---
decoder_inputs = layers.Input(shape=(SEQUENCE_LENGTH,), dtype="int64", name="decoder_inputs")
dec_emb_layer = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

# Define the Basic GRU Cell
decoder_cell = layers.GRUCell(LATENT_DIM)

# Wrap it with Attention
# Note: We pass the encoder outputs AND decoder embeddings to the call
attention_wrapper = DecoderRNNAttentionWrapper(cell_fn=decoder_cell, units=512)
decoder_outputs, attention_weights = attention_wrapper(
    [encoder_out, dec_emb],
    initial_state=encoder_state
)

# Final Output Layer
decoder_dense = layers.Dense(VOCAB_SIZE, activation="softmax")
final_outputs = decoder_dense(decoder_outputs)

# --- Model Compilation ---
attn_model = models.Model([encoder_inputs, decoder_inputs], final_outputs)

attn_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

attn_model.summary()

## 6️⃣ Training the Model

We train using Teacher Forcing (feeding the correct previous word to generate the next).

In [ ]:
history = attn_model.fit(
    train_ds,
    epochs=5, # Reduced for demonstration speed
    validation_data=val_ds
)

## 7️⃣ Visualization: Peeking into the "Brain"

The beauty of Attention is interpretability. We can extract the attention weights to see which English words the model focused on when generating a specific German word.

To do this, we need a separate **Inference Model** that outputs the attention weights.

In [ ]:
import matplotlib.pyplot as plt

# --- Define Inference Model for Visualization ---
# This model takes inputs and returns: Predictions, Attention Weights
vis_model = models.Model(
    inputs=[encoder_inputs, decoder_inputs], 
    outputs=[final_outputs, attention_weights]
)

def plot_attention(text_en, text_de_input):
    # Preprocess
    en_seq = en_vectorizer([text_en])
    de_seq = de_vectorizer([text_de_input])[:, :-1] # Remove end token for input
    
    # Predict
    preds, attn_weights = vis_model.predict([en_seq, de_seq])
    
    # attn_weights shape: (1, dec_len, enc_len)
    # Squeeze batch dimension
    attn_weights = attn_weights[0]
    
    # Get words for plotting
    en_words = text_en.split()
    de_words = text_de_input.split()[1:] # Skip 'sos'
    
    # Plot
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    
    # Slice the attention matrix to match sentence lengths
    # (The model pads to 20, but we only want to visualize real words)
    ax.matshow(attn_weights[:len(de_words), :len(en_words)], cmap='viridis')
    
    ax.set_xticklabels([''] + en_words, rotation=90)
    ax.set_yticklabels([''] + de_words)
    
    plt.xlabel('Source (English)')
    plt.ylabel('Target (German)')
    plt.show()

# Example Visualization
# Ideally, we generate the translation first, but for visualization here
# we will force feed a known pair to see alignment.
sample_en = "i am very happy"
sample_de = "sos ich bin sehr glücklich"

plot_attention(sample_en, sample_de)

### Interpretation of the Plot

If the model learned correctly, you should see a diagonal alignment:
* "ich" (I) should attend to "i".
* "bin" (am) should attend to "am".
* "sehr" (very) should attend to "very".

In complex translations where word order flips (e.g., German verbs moving to the end), the attention mechanism allows the model to jump to the correct encoder position effectively.

## 8️⃣ Chapter Summary

* **Limitations of Fixed Context:** Standard Seq2Seq models struggle with long sentences because they compress everything into one vector.
* **Attention Mechanism:** Allows the Decoder to create a dynamic context vector at every time step by taking a weighted sum of Encoder states.
* **Implementation:** We built a custom `DecoderRNNAttentionWrapper` because `K.rnn` allows us to define custom step logic involving alignment score calculation.
* **Alignment:** The attention weights $\alpha_{tj}$ represent the alignment between the target word at step $t$ and source word at step $j$.
* **Result:** Attention not only improves translation accuracy (BLEU scores) but makes the model interpretable via heatmaps.